In [7]:
#lista de parametros para API

import requests
import pandas as pd
import numpy as np

pd.set_option('future.no_silent_downcasting', True)

response = requests.get('https://servicodados.ibge.gov.br/api/v3/agregados')

if response.status_code == 200:
    data = response.json()

    resultado = pd.DataFrame({'categ_id':[], 'categ_nome':[], 'categ_unidade':[], 'categ_nivel':[], 'tabela_id':[],
       'tabela_classificacoes.id':[], 'tabela_classificacoes.nome':[],                      
       'var_id':[], 'var_nome':[], 'var_unidade':[], 'var_sumarizacao':[], 'tabela_id':[],
       'Tabela_nome':[], 'Tabela_URL':[], 'Tabela_pesquisa':[], 'Tabela_assunto':[],
       'Tabela_periodicidade.frequencia':[], 'Tabela_periodicidade.inicio':[],
       'Tabela_periodicidade.fim':[], 'Tabela_nivelTerritorial.Administrativo':[]})
    
    data_list = []

    for item in data:
        id = item['id']
        pesquisa = item['nome']
        for agregado in item['agregados']:
            tabela = agregado['id']
            nome_tabela = agregado['nome']
            data_list.append([id, pesquisa, tabela, nome_tabela])
            if 'SU' == item['id'] : df = pd.DataFrame(data_list, columns=['id', 'pesquisa', 'tabela', 'nome_tabela'])
    nada = df['tabela']

    lista = pd.DataFrame({'tabela':['2093']})
    print(lista)
    for item in lista:
        tab = item
        #print(IdTabela)
        enderecoT = 'https://servicodados.ibge.gov.br/api/v3/agregados/'+tab+'/metadados'
        print(enderecoT)
        responseT = requests.get(enderecoT)
        dataT = responseT.json()
        #print(dataT)
        df_categ = pd.json_normalize(dataT,
            record_path=['classificacoes','categorias'],
            meta=["id",["classificacoes","id"],["classificacoes","nome"]],
            meta_prefix='tabela_',
            record_prefix='categ_'
            )
       # print(df_categ)
                #Variaveis x classificacao

        df_var = pd.json_normalize(dataT,
            record_path=['variaveis'],
            meta=["id","nome","URL","pesquisa","assunto",["periodicidade","frequencia"],["periodicidade","inicio"],["periodicidade","fim"],["nivelTerritorial","Administrativo"]],
            meta_prefix='tabela_',
            record_prefix='var_'
                    )
        #nivel territorio x tabela

        df_nivel = pd.json_normalize(dataT,
            record_path=['nivelTerritorial','Administrativo'],
            meta=["id"],
            meta_prefix='tabela_',
            record_prefix='nivel_'
            )

        if df_categ.empty:
            dfm = df_var.merge(df_nivel,left_on='tabela_id',right_on='tabela_id')
        else:
            df1 = df_categ.merge(df_var, left_on='tabela_id',right_on='tabela_id')
            dfm = df1.merge(df_nivel, left_on='tabela_id',right_on='tabela_id')
        
        options = ['N1','N2','N3','N6'] 
        df = dfm.loc[dfm['nivel_0'].isin(options)] 
        #df = df.loc[:,'tabela_id']#,'var_id','tabela_classificacoes.id','categ_id','nivel_0']]
        #print(dfm)
        #print("muda/muda/muda")
        resultado = pd.concat([resultado,df])
        resultado = resultado.loc[:,['tabela_id','var_id','tabela_classificacoes.id','categ_id','nivel_0']]
        # resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']] = resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']].fillna(-1)
        # resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']] = resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']].astype(int)
        # resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']] = resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']].astype(str)
        # resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']] = resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']].replace('-1', np.nan)
        # resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']] = resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']].fillna('').astype(str)

    print("RESULTADO/RESULTADO/RESULTADO")    
    #print(resultado.dtypes)
    print(resultado)
    print(f"Request failed with status code {response.status_code}")

  tabela
0   2093
https://servicodados.ibge.gov.br/api/v3/agregados/tabela/metadados


KeyError: "Key 'id' not found. To replace missing values of 'id' with np.nan, pass in errors='ignore'"